In [1]:
%use dataframe
%use kandy
import org.jetbrains.kotlinx.dataframe.io.readCSV

val df = DataFrame.readCSV("../gclogs/pauses.csv")


// print dataframe
df.columnsCount()

16

In [2]:
df.rowsCount()


18622

In [3]:
df.countDistinct{type}


9

In [4]:
df.columnTypes()


[kotlin.String, kotlin.String, kotlin.String, kotlin.Double, kotlin.String, kotlin.Double, kotlin.Double, kotlin.Double, kotlin.Int, kotlin.Int, kotlin.Int, kotlin.Int, kotlin.Int, kotlin.Int, kotlin.Int, kotlin.Int]

In [5]:
df.head(5)


DataFrame: rowsCount = 5, columnsCount = 16

In [6]:
df.maxBy{duration}


DataRow: index = 13509, columnsCount = 16

In [7]:
df.groupBy{type}.sum()


DataFrame: rowsCount = 9, columnsCount = 13

In [8]:
df.filter { type == "Young" }.sortByDesc{duration}.head(5)


DataFrame: rowsCount = 5, columnsCount = 16

In [9]:
df.schema()


type: String
event_type: String
startedAt: String
duration: Double
cause: String
cpu_user: Double
cpu_kernel: Double
cpu_wallClock: Double
heap_occupancy_before_coll: Int
heap_occupancy_after_coll: Int
heap_size_before_coll: Int
heap_size_after_coll: Int
perm_or_meta_occupancy_before_coll: Int
perm_or_meta_occupancy_after_coll: Int
perm_or_meta_size_before_coll: Int
perm_or_meta_size_after_coll: Int

In [10]:
df.groupBy{cause}.count()


DataFrame: rowsCount = 5, columnsCount = 2

In [11]:
df.valueCounts(false) { type }.plot {
    bars {
        x(type)
        y("count")
        fillColor(type) {
            scale = categoricalColorHue()
            legend.type = legend.none()
        }
    }
    layout.title = "Pauses type count"
    layout.size = 950 to 500
}


In [12]:
df.describe()


DataFrame: rowsCount = 16, columnsCount = 12

In [13]:
df.select {type and duration}.groupBy{type}.sum()
        .add("max [s]"){df.filter { row -> it.type==row.type }.select("duration").max()}
        .add("mean [s]"){df.filter { row -> it.type==row.type }.select("duration").mean()}



DataFrame: rowsCount = 9, columnsCount = 4

In [14]:
df.select {type and duration}.groupBy{type}.aggregate {
    count() into "count"
    sum(duration) into "sum [s]"
    max(duration) into "max [s]"
    min(duration) into "min [s]"
    mean(duration) into "avg [s]"
 }


DataFrame: rowsCount = 9, columnsCount = 6

In [15]:
df.sum {duration}


97.10613200000051

In [16]:
df.select {event_type and duration}.groupBy{event_type}.sum()


DataFrame: rowsCount = 2, columnsCount = 2

In [17]:
df.select {type and duration}.groupBy{type}.sum()


DataFrame: rowsCount = 9, columnsCount = 2

In [18]:
df.filter { type=="Young" }.select{cause}.groupBy{cause}.count()


DataFrame: rowsCount = 2, columnsCount = 2